# Math of EOFs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## set plotting preferences and instantiate RNG
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})
rng = np.random.default_rng()

## Generate some data

In [ ]:
def sample_from_cov(Cxx, n_samples):
    """Draw 'n_samples' from specified covariance matrix"""

    ## check pos semi-definite
    assert np.all(np.diag(Cxx) > 0)
    assert np.allclose(Cxx.T, Cxx)

    ## cholesky decompose
    L = np.linalg.cholesky(Cxx)

    ## draw IID samples
    Y = rng.normal(size=(Cxx.shape[0], n_samples))

    return L @ Y


def plot_setup(ax):
    """modify plot to preferred style"""

    ## equal aspect ratio
    ax.set_aspect("equal")

    ## plot axes
    kwargs = dict(c="k", lw=0.5)
    ax.axhline(0, **kwargs)
    ax.axvline(0, **kwargs)

    ## hide tick labels
    ax.set_xticks([])
    ax.set_yticks([])

    return ax

## Plot data

In [ ]:
## specify covariance matrix
Cxx = np.array([[1.3, -0.7], [-0.7, 0.8]])

## draw some samples
X = sample_from_cov(Cxx, n_samples=40)

## plot samples
fig, ax = plt.subplots(figsize=(4, 4))
ax = plot_setup(ax)
ax.scatter(X[0], X[1])
plt.show()

## Geometric representation

### Pick a sample and pattern to plot

In [ ]:
## draw some samples
x = np.array([2, 1])
v = np.array([2, 2])

## get projection of x on v
xhat = (x.T @ v) / (v.T @ v) * v

## get reconstruction error
error = x - xhat

### Plot sample/pattern with error

In [ ]:
## labels for plotting
recon_label = r"Reconstruction $\left(\hat{\mathbf{x}}=\frac{\mathbf{p}^T\mathbf{x}}{\mathbf{p}^T\mathbf{p}}\mathbf{p}\right)$"
error_label = r"Error $\left(\mathbf{e}=\mathbf{x}-\hat{\mathbf{x}}\right)$"

## default args for plotting vector
arrow_kwargs = dict(head_width=0.12, width=0.02, length_includes_head=True)

## colors for plotting
colors = sns.color_palette()

## plot samples
fig, ax = plt.subplots(figsize=(4, 4))
ax.set_aspect("equal")
ax.set_xticks([])
ax.set_yticks([])
ax.scatter(0, 0, c="k", s=50, zorder=10)

for i, (u, label) in enumerate(
    zip([x, v], [r"Sample ($\mathbf{x}$)", r"Pattern ($\mathbf{p}$)"])
):

    ## plot sample and pattern
    arrow_kwargs = dict(head_width=0.12, width=0.015, length_includes_head=True)
    ax.arrow(0, 0, dx=u[1], dy=u[0], color=colors[i], **arrow_kwargs, label=label)


## plot projection
ax.arrow(
    0,
    0,
    dx=xhat[1],
    dy=xhat[0],
    color="k",
    ls="--",
    alpha=0.5,
    **arrow_kwargs,
    label=recon_label,
)

## plot error
ax.arrow(
    x=xhat[1],
    y=xhat[0],
    dx=error[1],
    dy=error[0],
    color="k",
    **arrow_kwargs,
    label=error_label,
)

ax.legend(prop=dict(size=10))
plt.show()

## Reconstruction error

The squared length of the vector $\mathbf{x}$ is $||\mathbf{x}||_2^2=\mathbf{x}^T\mathbf{x}$, and the length of the reconstruction vector is given by:
\begin{align}
    ||\hat{\mathbf{x}}||_2^2 &= \hat{\mathbf{x}}^T\hat{\mathbf{x}} = \left(\frac{\mathbf{p}^T\mathbf{x}}{\mathbf{p}^T\mathbf{p}}\mathbf{p}\right)^T\left(\frac{\mathbf{p}^T\mathbf{x}}{\mathbf{p}^T\mathbf{p}}\mathbf{p}\right)\\
    &= \frac{\left(\mathbf{p}^T\mathbf{x}\right)^2}{\mathbf{p}^T\mathbf{p}} \\
    &= \frac{\mathbf{p}^T\mathbf{x}\mathbf{x}^T\mathbf{p}}{\mathbf{p}^T\mathbf{p}}
\end{align}

Next, use the Pythagorean theorem to find the length of the error vector: 
\begin{align}
    \left| \left| \hat{\mathbf{x}} \right| \right|_2^2 + ||\mathbf{e}||_2^2 &= ||\mathbf{x}||_2^2\\
    \implies ||\mathbf{e}||_2^2 &= ||\mathbf{x}||_2^2 - \left| \left| \hat{\mathbf{x}} \right| \right|_2^2\\
    &= \mathbf{x}^T\mathbf{x} - \frac{\mathbf{p}^T\mathbf{x}\mathbf{x}^T\mathbf{p}}{\mathbf{p}^T\mathbf{p}}
\end{align}

## Choosing the "optimal" $\mathbf{p}$

Let's define the "optimal" $\mathbf{p}$ as the pattern which minimizes the reconstruction data for the entire dataset. The mean reconstruction error for the full dataset is given by:
\begin{align}
    \mathcal{R} &= \frac{1}{n}\sum^n_j \mathbf{x}_j^T\mathbf{x}_j - \frac{\mathbf{p}^T\mathbf{x}_j\mathbf{x}_j^T\mathbf{p}}{\mathbf{p}^T\mathbf{p}}
\end{align}

Note that the choice of $\mathbf{p}$ which minimizes $\mathcal{R}\propto \sum_j - \frac{\mathbf{p}^T\mathbf{x}_j\mathbf{x}_j^T\mathbf{p}}{\mathbf{p}^T\mathbf{p}}$ will *maximize* the squared projection length of the data onto the pattern, $\mathcal{P}=\frac{\sum_j \mathbf{p}^T\mathbf{x}_j\mathbf{x}_j^T\mathbf{p}}{\mathbf{p}^T\mathbf{p}}$! Therefore, without loss of generality, let's write down the optimization problem as:
\begin{align}
    \mathbf{p}^* &= \text{argmax}_\mathbf{p} \left(\frac{\sum_j \mathbf{p}^T\mathbf{x}_j\mathbf{x}_j^T\mathbf{p}}{\mathbf{p}^T\mathbf{p}}\right)
\end{align}

To find $\mathbf{p}^*$, start by rewriting the squared projection length:
\begin{align}
    \frac{\sum_j \mathbf{p}^T\mathbf{x}_j\mathbf{x}_j^T\mathbf{p}}{\mathbf{p}^T\mathbf{p}} &= \frac{\mathbf{p}^T\mathbf{C}_{xx}\mathbf{p}}{\mathbf{p}^T\mathbf{p}}
\end{align}

Where we set $\mathbf{C}_{xx}=\sum_j\mathbf{x}_j\mathbf{x}_j^T$. Next, write $\mathbf{C}_{xx}$ in terms of $\mathbf{X}$'s singular value decomposition. Letting $\mathbf{X}=\mathbf{V}\boldsymbol{\Lambda}\mathbf{U}^T$, we have $\mathbf{C}_{xx}=\mathbf{XX}^T=\mathbf{V}\boldsymbol{\Lambda}^2\mathbf{V}^T$, using the SVD property $\mathbf{U}^T\mathbf{U}=\mathbf{I}$ (note that the columns of $\mathbf{V}$ are eigenvectors of $\mathbf{C}_{xx}$; to see this, right multiply both sides by $\mathbf{V}$ and use the property $\mathbf{V}^T\mathbf{V}=\mathbf{I}$).  

Next, write $\mathbf{p}$ in terms of $\mathbf{V}$; that is $\mathbf{p}=\sum_i \alpha_i\mathbf{v}_i = \mathbf{V}\boldsymbol{\alpha}$. Substituting, we have:
\begin{align}
    \frac{\mathbf{p}^T\mathbf{C}_{xx}\mathbf{p}}{\mathbf{p}^T\mathbf{p}} &= \frac{\left(\boldsymbol{\alpha}^T \mathbf{V}^T\right)\left(\mathbf{V}\boldsymbol{\Lambda}^2\mathbf{V}^T\right)\mathbf{V}\boldsymbol{\alpha} }{\left(\boldsymbol{\alpha}^T \mathbf{V}^T\right)\left(\mathbf{V}\boldsymbol{\alpha}\right)}\\
    &= \frac{\boldsymbol{\alpha^T\Lambda^2\alpha}}{\boldsymbol{\alpha}^T\boldsymbol{\alpha}}\\
    &= \frac{\sum_j \lambda_j^2\alpha_j^2}{\sum_j\alpha_j^2}
\end{align}

Note the last quantity represents a weighted average of $\mathbf{C}_{xx}$'s eigenvalues, with the weight of the $i^{th}$ eigenvalue given by $\alpha_i^2$. To maximize the weighted average, set the weight for all non-leading eigenvalues to zero such that all the weight is on the largest eigenvalue. That is, set $\boldsymbol{\alpha}^T = \begin{bmatrix} 1 & 0 & \cdots & 0 \end{bmatrix}$. Then we have: $\mathbf{p}=\mathbf{V}\boldsymbol{\alpha}=\mathbf{v}_1$; that is, $\mathbf{p}$ is the leading eigenvector of $\mathbf{C}_{xx}$ (note $||\mathbf{v}_1||_2^2=1$ if obtained via SVD).

## Explained variance

Recall we decomposed each sample into a reconstruction and an error, with the square of the sample length equal to the square of the reconstruction length plus the square of the error length:
\begin{align}
    ||\mathbf{x}_j||^2 &= ||\hat{\mathbf{x}}_j||^2 + ||\mathbf{e}_j||^2
\end{align}
Summing over all samples, we have:
\begin{align}
    \sum_j||\mathbf{x}_j||^2 &= \sum_j ||\hat{\mathbf{x}}_j||^2 + \sum_j||\mathbf{e}_j||^2
\end{align}

The fraction of explained variance is the ratio of the length of the reconstruction term to the total term:
\begin{align}
    r &= \frac{\sum_j ||\hat{\mathbf{x}}_j||^2}{\sum_j ||\mathbf{x}_j||^2}\\
    &= \frac{\text{tr}\left[\left(\mathbf{pp}^T\mathbf{X}\right)^T\left(\mathbf{pp}^T\mathbf{X}\right)\right]}{\text{tr}\left(\mathbf{X}^T\mathbf{X}\right)}\\
    &= \frac{\text{tr}\left[\mathbf{X}^T\mathbf{pp}^T \mathbf{pp}^T\mathbf{X} \right]}{\text{tr}\left(\mathbf{X}^T\mathbf{X}\right)} =  \frac{\text{tr}\left[\mathbf{X}^T\mathbf{p} \mathbf{p}^T\mathbf{X} \right]}{\text{tr}\left(\mathbf{X}^T\mathbf{X}\right)}\\
    &= \frac{\mathbf{p}^T\mathbf{X} \mathbf{X}^T\mathbf{p}}{\text{tr}\left(\mathbf{X}\mathbf{X}^T\right)}
\end{align}
Where we used the commutative property of trace: $\text{tr}\left(\mathbf{A}\mathbf{B}^T\right)=\text{tr}\left(\mathbf{A}^T\mathbf{B}\right)$. Then, using the eigenvalue property of $\mathbf{p}$, we have $\mathbf{p}^T\mathbf{X} \mathbf{X}^T\mathbf{p} = \mathbf{p}^T\left(\lambda^2 \mathbf{p}\right)=\lambda^2$. For the numerator, write $\mathbf{XX}^T$ in terms of it's eigenvalue decomposition:
\begin{align}
    \text{tr}\left(\mathbf{XX}^T\right) &= \text{tr}\left(\sum_j\lambda_j^2 \mathbf{v}_j\mathbf{v}_j^T\right) = \sum_j\lambda_j^2\cdot\text{tr}\left( \mathbf{v}_j\mathbf{v}_j^T\right)= \sum_j\lambda_j^2\cdot\text{tr}\left( \mathbf{v}_j^T\mathbf{v}_j\right) = \sum_j\lambda_j^2
\end{align}
Therefore, the ratio of variance explained by the $i^{th}$ pattern is:
\begin{align}
    r &= \frac{\lambda_i^2}{\sum_j\lambda_j^2}
\end{align}